# Accessing satellite views 

In [ ]:
# ML libs
import pandas as pd
import numpy as np

# File manipulation
import os
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Graphic
import matplotlib.pyplot as plt
import plotly.express as px

# Environment variables
datasets_dir = os.environ.get('DATA_DIR_LOCAL')

In [ ]:
one_view_dir = "RPG/S2A_MSIL2A_20240412T105621_N0510_R094_T30TXP_20240412T165247.SAFE"
one_view_path = os.path.join(datasets_dir,one_view_dir)

# Accessing the bounding-box info of the satellite image

In [ ]:
one_view_path = os.path.join(one_view_path,os.listdir(one_view_path)[0])

In [ ]:
os.listdir(one_view_path)

In [ ]:
inspire_file = os.path.join(one_view_path, 'INSPIRE.xml')
with open(inspire_file) as fp:
    soup = BeautifulSoup(fp,'xml')

In [ ]:
soup.find('abstract').text.split()

In [ ]:
MTD_MSIL2A_file = os.path.join(one_view_path, 'MTD_MSIL2A.xml')
with open(MTD_MSIL2A_file) as fp:
    MTD_MSIL2A_soup = BeautifulSoup(fp,'xml')

In [ ]:
MTD_MSIL2A_coord_list = MTD_MSIL2A_soup.find('EXT_POS_LIST').text.split()

## Transform bouding-box info into geoshape data
<details><summary>
dictionary example
</summary>
```
{'type': 'FeatureCollection',
 'features': [{
    'type': 'Feature',
    'properties': {'GEO_ID': '0500000US01001',
        'STATE': '01',
        'COUNTY': '001',
        'NAME': 'Autauga',
        'LSAD': 'County',
        'CENSUSAREA': 594.436},
    'geometry': {
        'type': 'Polygon',
        'coordinates': [[[-86.496774, 32.344437],
         ...,
          [-86.496774, 32.344437]]]},
    'id': '01001'},
  {'type': 'Feature',
   'properties': {'GEO_ID': '0500000US01009',
        ...,
        'CENSUSAREA': 644.776},
   'geometry': {'type': 'Polygon',
        'coordinates': [[[-86.577799, 33.765316],
          ...,
          [-86.577799, 33.765316]]]},
   'id': '01009'},
    ...
}
```
</details>

In [ ]:
def list2polygoncoordinates(coord_list):
    return np.reshape(coord_list, (1, int(len(coord_list)/2),2))

In [ ]:
MTD_MSIL2A_coordinates = list2polygoncoordinates(MTD_MSIL2A_coord_list)
tmp_lat = MTD_MSIL2A_coordinates[:,:,0].copy()
MTD_MSIL2A_coordinates[:,:,0] =  MTD_MSIL2A_coordinates[:,:,1]
MTD_MSIL2A_coordinates[:,:,1] = tmp_lat
MTD_MSIL2A_coordinates

In [ ]:
img_center = MTD_MSIL2A_coordinates.astype(float).mean(axis=1)[0]

In [ ]:
geo_dict={'type': 'FeatureCollection',
          'features': [{
    'type': 'Feature',
    'properties': {},
    'geometry': {
        'type': 'Polygon',
        'coordinates': MTD_MSIL2A_coordinates},
    'id': '01'}]}

## Plotting on a map
A dataframe is needed to list the features to plot

In [ ]:
df = pd.DataFrame({'id':['01']})

In [ ]:
px.choropleth_mapbox(df, geojson=geo_dict, locations='id', 
                     mapbox_style="carto-positron",
                     zoom=6, 
                     opacity=0.2,
                     center = {"lon": img_center[0], "lat": img_center[1]}
                           )

## S3Path

In [ ]:
from s3path import S3Path

In [ ]:
bucket_path = S3Path("/eodata/Sentinel-2/MSI/L1C/2024/05/09/S2A_MSIL1C_20240509T105031_N0510_R051_T30TYN_20240509T125022.SAFE")

In [ ]:
bucket_path.exists()